In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
def habr_links(queries, pages=1):
    """
    Возвращает словарь всех ссылок, соответсвующих указанным запросам из списка queries.
    Ключами в словаре являются уникальные ссылки.
    Значениями в словаре являются один, или несколько соответствующих запросов.
    В параметре pages указывается количество страниц выдачи посика сайта https://habr.com/ru.
    По умолчанию pages=1.
    """
    all_links = {}
    
    for query in queries:
        params = {'q': query}

        for page in range(1, pages + 1):
            url = 'https://habr.com/ru/search/page' + str(page)
            time.sleep(0.1)
            soup = BeautifulSoup(requests.get(url, params=params).text)
            links = soup.find_all('a', class_='tm-article-snippet__title-link')
            
            if links == []:
                break

            for link in links:
                link = 'https://habr.com'+link.get('href')
                if link not in all_links:
                    all_links[link] = query
                else:
                    all_links[link] += ', '+query
    
    return all_links

In [3]:
def habr_df(all_links):
    """
    Возвращает датафрейм с информацией о постах на сайте https://habr.com/ru, соответствующих списку запросов.
    Столбцы датафрейма: запрос, дата, заголовок, ссылка, полный текст, и количество лайков.
    """
    df = pd.DataFrame()
    
    for link, query in all_links.items():
        
        try:
            time.sleep(0.1)
            
            soup = BeautifulSoup(requests.get(link).text)
            
            date = pd.to_datetime(soup.find('time').get('datetime'))
            title = soup.find('h1', class_='tm-article-snippet__title_h1').text
            full_text = soup.find('div', class_='article-formatted-body').text.strip()
            likes = soup.find('span', class_='tm-votes-meter__value').text.replace('+', '') # Не знаю есть ли на хабре дизлайки...
            
            row = pd.DataFrame([{'query': query, 'date': date, 'title': title, 'link': link, 'full_text': full_text, 'likes': likes}])
            df = pd.concat([df, row])
        except:
            pass # Сделано для обхода ссылок на несуществующие посты в поиске по Хабру, к примеру такие: https://habr.com/ru/company/epam_systems/news/t/589555/
        
    return df.reset_index(drop=True)

In [4]:
queries = ['python', 'анализ данных']
all_links = habr_links(queries, pages=2)
habr = habr_df(all_links)
print(len(habr))
habr[25:40]

77


,query,date,title,link,full_text,likes
25,"python, анализ данных",2021-12-10 10:15:02+00:00,Интенсив для повышения квалификации: как испол...,https://habr.com/ru/company/netologyru/news/t/...,"14 декабря в Нетологии пройдёт вебинар, посвящ...",4
26,python,2022-02-04 11:04:02+00:00,В Visual Studio Code добавили новые возможност...,https://habr.com/ru/news/t/649717/,В блоге Microsoft сообщили о нововведениях пла...,9
27,python,2022-05-24 07:57:47+00:00,Дайджест Слёрма: тест на уровень кунг-фу по Py...,https://habr.com/ru/company/southbridge/news/t...,Сделали для вас подборку свежих статей и выгод...,9
28,python,2021-09-22 05:03:44+00:00,Открытый вебинар «The Python Packaging»,https://habr.com/ru/news/t/579264/,23 сентября в 16:30 в рамках цикла открытых те...,3
29,python,2021-10-18 07:08:07+00:00,Intel оптимизировала библиотеку Numpy под AVX-...,https://habr.com/ru/company/itsumma/news/t/584...,Расположение векторного регистрового файла (жё...,39
30,python,2021-10-20 13:43:08+00:00,Новый хакатон от СберЗвука для Python-разработ...,https://habr.com/ru/news/t/584584/,"Привет, Хабр! Если вы хотите проверить свои си...",3
31,python,2022-06-27 13:57:38+00:00,В стороннем официальном репозитории Python был...,https://habr.com/ru/news/t/673668/,В официальном стороннем репозитории Python был...,7
32,python,2021-12-08 20:20:41+00:00,"Открытый семинар: FastAPI, или как быстро доба...",https://habr.com/ru/news/t/594325/,9 декабря в 16:30 Виктор Бебнев (Xperience AI)...,2
33,python,2022-04-18 03:29:27+00:00,Онлайн-митап от руководителя практики Python U...,https://habr.com/ru/company/usetech/news/t/661...,"В конце марта Мстислав Казаков, руководитель п...",0
34,python,2022-07-25 15:07:01+00:00,Бесплатный курс по бэкенд-разработке на Python...,https://habr.com/ru/company/southbridge/news/t...,"Как людей учат разработке? Вовлекают, дают инт...",4
